# Ax Managed Loop Example

Ax' managed loop API is meant to allow for reduction of optimization to a single call and for CLI compatibility. This API is a work in progress.

In [1]:
from ax.service.managed_loop import OptimizationLoop, OptimizationPlan
from ax.metrics.branin import branin

In [2]:
def branin_evaluation_function(
    parameterization, # dict of parameter names to values of those parameters
    weight=None, # evaluation function signature requires a weight argument
):
    if any(
        param_name not in parameterization.keys() for param_name in ["x1", "x2"]
    ):
        raise ValueError("Parametrization does not contain x1 or x2")
    x1, x2 = parameterization["x1"], parameterization["x2"]
    return {"branin": (branin(x1, x2), 0.0), "constrained_metric": (-branin(x1, x2), 0.0)}

The setup for the loop is similar to that of the service API and will be fully compatible with JSON. Currently, the optimization algorithm is chosen under the hood based on the properties of the problem search space, but it will be exposed for choice shortly.

In [3]:
loop = OptimizationLoop.with_evaluation_function(
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [-5.0, 10.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {  
            "name": "x2",  # We'll omit the optional arguments for this parameter.
            "type": "range",
            "bounds": [0.0, 10.0],
        },
    ],
    experiment_name="test",
    objective_name="branin",
    evaluation_function=branin_evaluation_function,
    minimize=True,  # Optional, defaults to False.
    parameter_constraints=["x1 + x2 <= 20"],  # Optional.
    outcome_constraints=["constrained_metric <= 10"],  # Optional.
    optimization_plan=OptimizationPlan(total_iterations=10),  # Optional.
)

[INFO 04-05 08:45:05] ax.service.utils.dispatch: Using Bayesian Optimization generation strategy. Iterations after 5 will take longer to generate due to model-fitting.


The optimization stage is a single call:

In [4]:
loop.full_run().get_best_point()

[INFO 04-05 08:45:05] ax.service.managed_loop: Started full optimization with 10 steps.
[INFO 04-05 08:45:05] ax.service.managed_loop: Running optimization step 1...
[INFO 04-05 08:45:05] ax.service.managed_loop: Running optimization step 2...
[INFO 04-05 08:45:05] ax.service.managed_loop: Running optimization step 3...
[INFO 04-05 08:45:05] ax.service.managed_loop: Running optimization step 4...
[INFO 04-05 08:45:05] ax.service.managed_loop: Running optimization step 5...
[INFO 04-05 08:45:05] ax.service.managed_loop: Running optimization step 6...
[INFO 04-05 08:45:48] ax.service.managed_loop: Running optimization step 7...
[INFO 04-05 08:46:20] ax.service.managed_loop: Running optimization step 8...
[INFO 04-05 08:46:54] ax.service.managed_loop: Running optimization step 9...
[INFO 04-05 08:47:39] ax.service.managed_loop: Running optimization step 10...


{'x1': -4.999999999999748, 'x2': 3.908664869277646e-14}